In [1]:
%pip install fastmcp pandas langchain openai uvicorn --upgrade --no-cache-dir

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 29.8 MB/s eta 0:00:00
  Attempting uninstall: openai
    Found existing installation: openai 2.6.1
    Uninstalling openai-2.6.1:
      Successfully uninstalled openai-2.6.1
  Attempting uninstall: langchain-core━━━━━━━━━━ 0/3 [openai]
    Found existing installation: langchain-core 1.1.1m0/3 [openai]
    Uninstalling langchain-core-1.1.1:━━━━━━ 0/3 [openai]
      Successfully uninstalled langchain-core-1.1.132m0/3 [openai]
  Attempting uninstall: langchain0m━━━━━━━━━━━━━━━━━━━━━━━━━━ 1/3 [langchain-core]
    Found existing installation: langchain 1.1.2━━━━━━━━━━━━━━ 1/3 [langchain-core]
    Uninstalling langchain-1.1.2:90m━━━━━━━━━━━━━━━━━━━━━━━━━━ 1/3 [langchain-core]
      Successfully uninstalled langchain-1.1.2━━━━━━━━━━━━━━━━ 1/3 [langchain-core]
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3/3 [langchain]0m [langchain-core]

[notice] A new release of pip is available: 25.1.1 -> 25.3
[notice] To update, run: pip install --upgr

In [2]:
from fastmcp import Client
from fastmcp.client import PythonStdioTransport
import asyncio

async def main():
    # Create a client using the PythonStdioTransport
    # Create a loop to run the client

    transport = PythonStdioTransport("mcp_server_fastmcp.py")
    # client = Client(transport)
    async with Client(transport) as client:
        tools = await client.list_tools()
        print("Tools:", [t.name for t in tools])

await main()

Tools: ['summarize', 'query']


In [3]:
def decide_tool(text: str):
    text = text.lower()
    if "summarize" in text or "overview" in text:
        return "summarize", {}
    if "west" in text:
        return "query", {"expr": "region == 'West' and sales > 1500"}
    return "summarize", {}

async def run_agent(user_input, client):
    tool, params = decide_tool(user_input) #"summarize", {}
    #tool = "summarize"
    #params = {}
    print(f"Agent decided to use '{tool}'")

    # API for fastmcp 2.12.5
    result = await client.call_tool(tool, params)
    # result = await client.call_tool("query", params)

    print("Result:", result, "\n")

In [4]:
# create a connection to your MCP server
transport = PythonStdioTransport("mcp_server_fastmcp.py")

async with Client(transport) as client:
    await run_agent("Give me a summary of the dataset", client)

Agent decided to use 'summarize'
Result: CallToolResult(content=[TextContent(type='text', text='{"rows":10,"columns":["order_id","region","sales","quarter","rep"],"numeric_stats":{"order_id":{"count":10.0,"unique":"","top":"","freq":"","mean":105.5,"std":3.0276503540974917,"min":101.0,"25%":103.25,"50%":105.5,"75%":107.75,"max":110.0},"region":{"count":10,"unique":4,"top":"West","freq":4,"mean":"","std":"","min":"","25%":"","50%":"","75%":"","max":""},"sales":{"count":10.0,"unique":"","top":"","freq":"","mean":1235.0,"std":536.4751210965477,"min":600.0,"25%":825.0,"50%":1175.0,"75%":1450.0,"max":2200.0},"quarter":{"count":10,"unique":4,"top":"Q1","freq":3,"mean":"","std":"","min":"","25%":"","50%":"","75%":"","max":""},"rep":{"count":10,"unique":10,"top":"Alex","freq":1,"mean":"","std":"","min":"","25%":"","50%":"","75%":"","max":""}}}', annotations=None, meta=None)], structured_content={'rows': 10, 'columns': ['order_id', 'region', 'sales', 'quarter', 'rep'], 'numeric_stats': {'order_

In [5]:
# create a connection to your MCP server
transport = PythonStdioTransport("mcp_server_fastmcp.py")

async with Client(transport) as client:
    await run_agent("Give me a summary of the dataset", client)
    await run_agent("Show West region sales > 1500", client)

Agent decided to use 'summarize'
Result: CallToolResult(content=[TextContent(type='text', text='{"rows":10,"columns":["order_id","region","sales","quarter","rep"],"numeric_stats":{"order_id":{"count":10.0,"unique":"","top":"","freq":"","mean":105.5,"std":3.0276503540974917,"min":101.0,"25%":103.25,"50%":105.5,"75%":107.75,"max":110.0},"region":{"count":10,"unique":4,"top":"West","freq":4,"mean":"","std":"","min":"","25%":"","50%":"","75%":"","max":""},"sales":{"count":10.0,"unique":"","top":"","freq":"","mean":1235.0,"std":536.4751210965477,"min":600.0,"25%":825.0,"50%":1175.0,"75%":1450.0,"max":2200.0},"quarter":{"count":10,"unique":4,"top":"Q1","freq":3,"mean":"","std":"","min":"","25%":"","50%":"","75%":"","max":""},"rep":{"count":10,"unique":10,"top":"Alex","freq":1,"mean":"","std":"","min":"","25%":"","50%":"","75%":"","max":""}}}', annotations=None, meta=None)], structured_content={'rows': 10, 'columns': ['order_id', 'region', 'sales', 'quarter', 'rep'], 'numeric_stats': {'order_